# Extracting HuBERT Hidden Representations

In [6]:
import librosa
from transformers import HubertForCTC, Wav2Vec2Processor
import numpy as np
import glob
from helper_scripts.TenseLax import TenseLax

# 1) Import Audio Files

In [7]:
# DESIRED_PHONEMES = *TenseLax.getPairs()
DESIRED_PHONEMES = [vowel for vowel_pair in TenseLax.getPairs() for vowel in vowel_pair]
print (DESIRED_PHONEMES)

['iy', 'ih', 'eh', 'ey', 'eh', 'ae', 'ow', 'ao', 'uw', 'uh']


In [ ]:
DATASET_PATH = 
phoneme_paths = glob.glob()